In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 51.8 MB/s eta 0:00:00:00:0100:01


In [2]:
import duckdb
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

In [4]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mol1 = con.query(f"""(SELECT DISTINCT(buildingblock1_smiles) as buildingblock1_smiles
                        FROM parquet_scan('{train_path}')
                        )""").df()

mol2 = con.query(f"""(SELECT DISTINCT(buildingblock2_smiles) as buildingblock1_smiles
                        FROM parquet_scan('{train_path}')
                        )""").df()

mol3 = con.query(f"""(SELECT DISTINCT(buildingblock3_smiles) as buildingblock1_smiles
                        FROM parquet_scan('{train_path}')
                        )""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [9]:
mol1.loc[0, 'buildingblock1_smiles']

'C=CC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O'

In [11]:
mol2.loc[0, 'buildingblock1_smiles']

'Nc1ncc(Cl)c(Cl)n1'

In [13]:
mol3.loc[0, 'buildingblock1_smiles']

'Br.Br.NCC1CCCN1c1cccnn1'

In [17]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        LIMIT 100
                        )""").df()

con.close()

In [18]:
mols.loc[0, 'molecule_smiles']

'C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1'

In [20]:
mols.loc[0, 'buildingblock1_smiles']

'C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21'

In [22]:
mols.loc[0, 'buildingblock2_smiles']

'C#CCOc1ccc(CN)cc1.Cl'

In [23]:
mols.loc[0, 'buildingblock3_smiles']

'Br.Br.NCC1CCCN1c1cccnn1'

In [9]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

duplicates = con.query(f"""(
                    WITH normalized_data AS (
                    SELECT id, 
        LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AS block1,
        CASE
            WHEN buildingblock1_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock2_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock3_smiles
            WHEN buildingblock1_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock3_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock2_smiles
            WHEN buildingblock2_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock3_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock1_smiles
            WHEN buildingblock2_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock1_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock3_smiles
            WHEN buildingblock3_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock1_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock2_smiles
            ELSE buildingblock1_smiles
        END AS block2,
        GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AS block3
    FROM parquet_scan('{test_path}')
    WHERE protein_name = 'BRD4'
)

SELECT 
    block1,
    block2,
    block3,
    COUNT(*) AS occurrence_count
FROM normalized_data
GROUP BY 
    block1,
    block2,
    block3
HAVING 
    COUNT(*) > 1)""").df()

con.close()

In [10]:
duplicates

,block1,block2,block3,occurrence_count


In [11]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

duplicates = con.query(f"""(
                    WITH normalized_data AS (
                    SELECT id, 
        LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AS block1,
        CASE
            WHEN buildingblock1_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock2_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock3_smiles
            WHEN buildingblock1_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock3_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock2_smiles
            WHEN buildingblock2_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock3_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock1_smiles
            WHEN buildingblock2_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock1_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock3_smiles
            WHEN buildingblock3_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock1_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock2_smiles
            ELSE buildingblock1_smiles
        END AS block2,
        GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AS block3
    FROM parquet_scan('{test_path}')
    WHERE protein_name = 'HSA'
)

SELECT 
    block1,
    block2,
    block3,
    COUNT(*) AS occurrence_count
FROM normalized_data
GROUP BY 
    block1,
    block2,
    block3
HAVING 
    COUNT(*) > 1)""").df()

con.close()

In [12]:
duplicates

,block1,block2,block3,occurrence_count


In [13]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

duplicates = con.query(f"""(
                    WITH normalized_data AS (
                    SELECT id, 
        LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AS block1,
        CASE
            WHEN buildingblock1_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock2_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock3_smiles
            WHEN buildingblock1_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock3_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock2_smiles
            WHEN buildingblock2_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock3_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock1_smiles
            WHEN buildingblock2_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock1_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock3_smiles
            WHEN buildingblock3_smiles = LEAST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AND buildingblock1_smiles = GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) THEN buildingblock2_smiles
            ELSE buildingblock1_smiles
        END AS block2,
        GREATEST(buildingblock1_smiles, buildingblock2_smiles, buildingblock3_smiles) AS block3
    FROM parquet_scan('{test_path}')
    WHERE protein_name = 'sEH'
)

SELECT 
    block1,
    block2,
    block3,
    COUNT(*) AS occurrence_count
FROM normalized_data
GROUP BY 
    block1,
    block2,
    block3
HAVING 
    COUNT(*) > 1)""").df()

con.close()

In [14]:
duplicates

,block1,block2,block3,occurrence_count
